In [22]:
import os

# Set Postgres environment variables
os.environ["POSTGRES_HOST"] = "localhost"  # your host
os.environ["POSTGRES_PORT"] = "5432"  # your port
os.environ["POSTGRES_DB"] = "routes_db"  # your database name
os.environ["POSTGRES_USER"] = "postgres"  # your username
os.environ["POSTGRES_PASSWORD"] = "password"  # your password

In [23]:
connection_str = (
    f"postgresql://{os.environ['POSTGRES_USER']}:{os.environ['POSTGRES_PASSWORD']}"
    f"@{os.environ['POSTGRES_HOST']}:{os.environ['POSTGRES_PORT']}/{os.environ['POSTGRES_DB']}"
)

In [24]:
from semantic_router.index.postgres import PostgresIndex

pg_index = PostgresIndex(connection_string=connection_str, index_name="new_table")

In [25]:
pg_index.setup_index()

In [26]:
pg_index._get_table_name()

'semantic_router_new_table'

In [27]:
pg_index._check_embeddings_dimensions()

True

In [28]:
import os
from getpass import getpass
from semantic_router.encoders import OpenAIEncoder

# get at platform.openai.com
os.environ["OPENAI_API_KEY"] = os.environ.get("OPENAI_API_KEY") or getpass(
    "Enter OpenAI API key: "
)

encoder = OpenAIEncoder()

In [29]:
# Import necessary modules
from semantic_router import Route

# Define routes to guide the chatbot's responses
politics = Route(
    name="politics",
    utterances=[
        "isn't politics the best thing ever",
        "why don't you tell me about your political opinions",
        "don't you just love the president",
        "don't you just hate the president",
        "they're going to destroy this country!",
        "they will save the country!",
    ],
)

# Define a chitchat route for general conversations
chitchat = Route(
    name="chitchat",
    utterances=[
        "how's the weather today?",
        "how are things going?",
        "lovely weather today",
        "the weather is horrendous",
        "let's go to the chippy",
    ],
)

# Combine both routes into a single list
routes = [politics, chitchat]

In [30]:
# Import the SemanticRouter class
from semantic_router.routers import SemanticRouter

# Initialize the SemanticRouter with the encoder, routes, and index
rl = SemanticRouter(encoder=encoder, routes=routes, index=pg_index, auto_sync="local")

2025-06-19 14:48:14 - httpx - INFO - _client.py:1013 - _send_single_request() - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-06-19 14:48:14 - semantic_router.utils.logger - WARNING - postgres.py:557 - _write_config() - No config is written for PostgresIndex.


In [31]:
# List the names of the defined routes
rl.list_route_names()

['politics', 'chitchat']

In [32]:
# Check the total number of entries in the index
len(rl.index)

11

In [33]:
# Get all records for the 'politics' route
rl.index._get_route_ids(route_name="politics")

['politics#c69244df-bcba-5bb1-83c2-94b88af3f2a0',
 'politics#c24f2d13-c2d5-5ddf-9169-03646cc3dad0',
 'politics#af3cd8c8-defb-5940-b66a-d59e8dbbada7',
 'politics#f248c2fa-4ab0-5bf9-8678-b25f7945705a',
 'politics#5f21f564-3d1e-58d9-a0b5-9e0c79e33d72',
 'politics#ce06418e-133f-5484-83fb-d8d3f136fc5d']

In [34]:
# Query the route layer with a statement related to politics
rl("I like voting. What do you think about the president?")

2025-06-19 14:48:14 - httpx - INFO - _client.py:1013 - _send_single_request() - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


RouteChoice(name='politics', function_call=None, similarity_score=0.4705034316574012)

In [35]:
# Query the route layer with a chitchat statement
rl("how's the weather today?").name

2025-06-19 14:48:15 - httpx - INFO - _client.py:1013 - _send_single_request() - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


'chitchat'

In [36]:
# Query the route layer with another chitchat statement
rl("where are you?").name

2025-06-19 14:48:15 - httpx - INFO - _client.py:1013 - _send_single_request() - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


In [37]:
# Check the total number of entries in the index before deletion
len(rl.index)

11

In [38]:
# Delete the 'chitchat' route and check the index length after deletion
import time

rl.delete(route_name="chitchat")
time.sleep(1)
len(rl.index)

2025-06-19 14:48:15 - semantic_router.utils.logger - WARNING - base.py:316 - _read_config() - This method should be implemented by subclasses.
2025-06-19 14:48:15 - semantic_router.utils.logger - WARNING - base.py:316 - _read_config() - This method should be implemented by subclasses.
2025-06-19 14:48:15 - semantic_router.utils.logger - WARNING - postgres.py:557 - _write_config() - No config is written for PostgresIndex.


6

In [39]:
# Attempt to query the deleted 'chitchat' route
print(rl("how's the weather today?").name)

2025-06-19 14:48:16 - httpx - INFO - _client.py:1013 - _send_single_request() - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


None


In [40]:
# Get all the current routes and their utterances
rl.index.get_routes()

[Route(name='politics', utterances=["don't you just hate the president", "don't you just love the president", "isn't politics the best thing ever", 'they will save the country!', "they're going to destroy this country!", "why don't you tell me about your political opinions"], description=None, function_schemas=None, llm=None, score_threshold=None, metadata={})]

In [41]:
# Describe the index to get details like type, dimensions, and total vector count
rl.index.describe()

IndexConfig(type='postgres', dimensions=1536, vectors=6)

need to add a unlock method

In [42]:
rl.index.close()